<a href="https://colab.research.google.com/github/Mohona77/Audio-Classification-SER-/blob/main/BanglaSER_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
pip install transformers datasets evaluate

In [33]:
from huggingface_hub import notebook_login

notebook_login()

In [34]:
from transformers import AutoFeatureExtractor

feature_extractor = AutoFeatureExtractor.from_pretrained("facebook/wav2vec2-base")

In [35]:
%%capture
!pip install gdown

In [36]:
import os
import zipfile
from google.colab import files
import time
import json
import librosa
from datasets import Dataset, DatasetDict, Audio
import glob
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [ ]:
%%capture
!gdown --id 1fZoWnL0neesTfj3l1kQRNN5O0Sfm3Bjl
time.sleep(5)

!unzip /content/BanglaSER.zip

In [4]:
import os
import librosa
import json
from sklearn.model_selection import train_test_split

# Emotion mapping based on your dataset description
emotion_map = {
    1: "Happy",
    2: "Sad",
    3: "Angry",
    4: "Surprise",
    5: "Neutral"
}

def parse_filename(filename):
    """Parses the filename to extract the emotion label."""
    parts = filename.split('-')
    if len(parts) >= 7:
        # Extract the emotion label from the 3rd part
        emotion_label = int(parts[2])  # Emotion label is in the 3rd part
        return emotion_map.get(emotion_label, "Unknown")
    return None

def create_json_data(directory):
    """Creates a JSON data structure from all subfolders in a given directory."""
    json_data = []
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith('.wav'):
                emotion_label = parse_filename(file)
                if emotion_label is not None and emotion_label != "Unknown":
                    file_path = os.path.join(root, file)
                    audio, sr = librosa.load(file_path, sr=None)
                    audio_length = librosa.get_duration(y=audio, sr=sr)
                    data_entry = {
                        "Id": file.split('.')[0],
                        "Emotion Label": emotion_label,
                        "Length": audio_length,
                        "Audio Array": audio.tolist(),
                        "WAV Link": file_path
                    }
                    json_data.append(data_entry)
    return json_data


def filter_data(json_data, remove_emotion_label):
    """Filters out data with the specified emotion label."""
    return [entry for entry in json_data if entry["Emotion Label"] != remove_emotion_label]

def split_data(json_data, seed=32):
    """Splits the data into train and test sets with an 80-20% distribution."""
    train, test = train_test_split(json_data, test_size=0.2, random_state=seed)
    return train, test


In [ ]:

# Directory path for your audio files
directory_path = '/content/t9h6p943xy-5/BanglaSER'

# Step 1: Create JSON data from audio files
print("Creating JSON data...")
json_data = create_json_data(directory_path)

# Step 2: Filter out 'surprise' emotion label (assuming label is "Surprise")
print("Filtering out 'surprise' emotion...")
filtered_data = filter_data(json_data, remove_emotion_label="Surprise")

# Step 3: Split the filtered data into train and test sets
print("Splitting data into train and test sets...")
train_data, test_data = split_data(filtered_data)

# Step 4: Save the split data to JSON files
print("Saving train and test JSON files...")
with open('train_data.json', 'w') as train_file:
    json.dump(train_data, train_file, indent=4)

with open('test_data.json', 'w') as test_file:
    json.dump(test_data, test_file, indent=4)

print("Train and test JSON files created successfully.")


Creating JSON data...
